In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
tf.__version__

'2.10.0'

In [2]:
listing_data = pd.read_csv('bukken_data_coordinates_fukuoka.csv', index_col=0)
listing_data.reset_index(inplace = True, drop = True)
print(listing_data.shape)
listing_data[372:374]

(8832, 18)


,Link,Full address,Latitude,Longitude,Prefecture,City,Ward,Neighborhood,Floor plan,Area m2,Balcony m2,Construction year,Floor,Highest floor,Building type,Closest station (min),Number of stations,Price
372,https://suumo.jp/ms/chuko/fukuoka/sc_kurume/nc...,福岡県久留米市善導寺町与田,33.321331,130.603592,福岡,久留米,善導寺町与,善導寺町与田,3LDK,70.08,0.00,1997,5,6.0,陸屋根,9.0,1.0,1380万円
373,https://suumo.jp/ms/chuko/fukuoka/sc_kitakyush...,福岡県北九州市小倉北区熊本３,33.867798,130.893738,福岡,北九州,小倉北,熊本３,5DK,87.80,9.39,1991,4-5,4.0,SRC14階地下,15.0,3.0,1380万円


# Cleaning data

## Remove uncessary columns for model

In [3]:
#listing_data = listing_data.drop(['Link', 'Full address', 'Prefecture', 'City', 'Ward', 'Neighborhood'], axis=1)
listing_data = listing_data.drop(['Full address', 'Prefecture', 'City', 'Ward', 'Neighborhood', 'Floor plan', 'Building type'], axis=1)
listing_data.tail(1)

,Link,Latitude,Longitude,Area m2,Balcony m2,Construction year,Floor,Highest floor,Closest station (min),Number of stations,Price
8831,https://suumo.jp/ms/chuko/fukuoka/sc_fukuokash...,33.662354,130.418076,107.74,11.92,2019,43,46.0,32.0,2.0,1億円


## Floor plan

### Simplify categories

In [ ]:
listing_data['Floor plan'].unique()

In [ ]:
listing_data['Floor plan'] = listing_data['Floor plan'].replace({'1ワンルーム', 'ワンルーム'}, '1R')

In [ ]:
# listing_data['Type'] = test.str.replace('（納戸）', '')
listing_data['Floor plan'] = listing_data['Floor plan'].str.replace('\+[0-9]?S（納戸）', '', regex = True)

In [ ]:
listing_data['Floor plan'].unique()

In [ ]:
types = list(listing_data['Floor plan'].unique())
types.sort()
types

In [ ]:
#types.extend(['1DK+S','1DK+2S','1LDK+2S','1LK','1LK+S','1LK+2S', '2DK+2S', '2LK+S','2LK+2S', '3DK+2S', '3LK', '3LK+2S', '4DK+S', '4LK', '4LK+S','4LK+2S', '5DK+2S', '5LK', '5LK+S','5LK+2S','5LDK+2S'])
types.extend(['1LK', '3LK','4LK', '5LK'])
types.sort()
types

### One hot encoding

In [ ]:
mapping = dict()

In [ ]:
for i, b_type in enumerate(types):
    mapping[b_type]=b_type

In [ ]:
listing_data['Floor plan'] = listing_data['Floor plan'].map(mapping)
listing_data = pd.get_dummies(listing_data, columns=['Floor plan'], prefix='', prefix_sep='')
listing_data.head()

## Building type

In [ ]:
listing_data['Building type'].unique()

In [ ]:
listing_data['Building type'] = listing_data['Building type'].str.replace(r'\d+階', '', regex = True)
listing_data['Building type'] = listing_data['Building type'].str.replace('ＳＲＣ', 'SRC')
listing_data['Building type'] = listing_data['Building type'].str.replace('ＲＣ', 'RC')
listing_data['Building type'] = listing_data['Building type'].str.replace('地下', '')
listing_data['Building type'] = listing_data['Building type'].str.replace('造', '')
listing_data['Building type'] = listing_data['Building type'].str.replace('鉄骨', 'S')
listing_data['Building type'] = listing_data['Building type'].str.replace('RC一部RC', 'RC')
listing_data['Building type'] = listing_data['Building type'].str.replace('SRC一部SRC', 'SRC')
listing_data['Building type'] = listing_data['Building type'].str.replace('PC一部PC', 'PC')
listing_data['Building type'] = listing_data['Building type'].str.replace('鉄筋コンクリート', 'RC')
listing_data['Building type'] = listing_data['Building type'].str.replace('一部', '+')
listing_data['Building type'] = listing_data['Building type'].str.replace('、', '+')
listing_data['Building type'] = listing_data['Building type'].str.replace('・', '+')
listing_data['Building type'] = listing_data['Building type'].str.replace('-', '+')
listing_data['Building type'] = listing_data['Building type'].str.replace('-', '+')
listing_data['Building type'].unique()

In [ ]:
b_mapping = dict()
b_types = list(listing_data['Building type'].unique())
for i, e in enumerate(b_types):
    b_mapping[e]=e

In [ ]:
listing_data['Building type'] = listing_data['Building type'].map(b_mapping)
listing_data = pd.get_dummies(listing_data, columns=['Building type'], prefix='', prefix_sep='')
listing_data.head(3)

## Price

In [ ]:
listing_data['Price'].describe().transpose()

In [4]:
def remove_thousands(price):
    try:
        man = price.index('万')
        return price[:man]
    except:
        return price

listing_data['Price'] = listing_data['Price'].str.replace('億円', '0000')
listing_data['Price'] = listing_data['Price'].str.replace('億', '')
listing_data['Price'] = listing_data['Price'].str.replace('円', '')
listing_data['Price'] = listing_data['Price'].apply(remove_thousands)
listing_data['Price'] = listing_data['Price'].astype('int64')

In [5]:
listing_data.iloc[listing_data['Price'].idxmax()]['Link']

'https://suumo.jp/ms/chuko/fukuoka/sc_fukuokashichuo/nc_98026235/'

In [6]:
listing_data['Price'].describe()

count     8832.000000
mean      2286.141418
std       1487.224437
min        100.000000
25%       1580.000000
50%       2099.000000
75%       2749.000000
max      45000.000000
Name: Price, dtype: float64

## Floor

In [7]:
listing_data['Floor'].unique()

array(['2', '5', '4', '3', '8', '6', '1', '11', '7', '9', '12', '15',
       '13', '10', '14', '16', '20', '33', '22', '37', '36', '43', '19',
       '4-5', '6-8', 'B1', '1-2', '1-3', '35', '31', '21', '25', '41',
       'A', '7-8', '9-10', '1-B1', '24', '6-7', '34', '17', 'B1-1', '23',
       '8-9', '28', '27', '18', '26', '44'], dtype=object)

In [8]:
def cleanup(x):
    x=str(x)
    try: 
        return x[0:x.index('-')]
    except ValueError:
        return x


#listing_data['Floor'] = listing_data['Floor'].str.replace('-(B)?[0-9][0-9]?', '', regex=True)
#listing_data['Floor'] = listing_data['Floor'].apply(lambda x: try: x[:x.index('-')] except ValueError: x)
listing_data['Floor'] = listing_data['Floor'].apply(cleanup)
listing_data['Floor'] = listing_data['Floor'].apply(lambda x: -1 if(x[0]=='B') else x)
listing_data = listing_data[listing_data['Floor']!='A']
listing_data['Floor'] = listing_data['Floor'].astype('int8')
listing_data['Floor'].unique()

array([ 2,  5,  4,  3,  8,  6,  1, 11,  7,  9, 12, 15, 13, 10, 14, 16, 20,
       33, 22, 37, 36, 43, 19, -1, 35, 31, 21, 25, 41, 24, 34, 17, 23, 28,
       27, 18, 26, 44], dtype=int8)

In [9]:
listing_data['Closest station (min)'] = listing_data['Closest station (min)'].astype('uint8')
listing_data['Number of stations'] = listing_data['Number of stations'].astype('uint8')
listing_data['Number of stations'] = listing_data['Number of stations'].astype('uint8')
listing_data['Highest floor'] = listing_data['Highest floor'].astype('uint8')

# Model creation

In [10]:
listing_data = listing_data.drop('Link', axis=1)
listing_data.dtypes

Latitude                 float64
Longitude                float64
Area m2                  float64
Balcony m2               float64
Construction year          int64
Floor                       int8
Highest floor              uint8
Closest station (min)      uint8
Number of stations         uint8
Price                      int64
dtype: object

## Train and test set

In [11]:
train_dataset = listing_data.sample(frac=0.8, random_state=0)
test_dataset = listing_data.drop(train_dataset.index)

In [12]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Price')
test_labels = test_features.pop('Price')

In [13]:
train_dataset.describe().transpose()[['mean', 'std']]

,mean,std
Latitude,33.634637,0.143737
Longitude,130.513562,0.189524
Area m2,72.287837,16.592785
Balcony m2,12.771101,8.773028
Construction year,1992.368063,11.530991
Floor,5.172282,3.849927
Highest floor,8.857305,4.567895
Closest station (min),9.272225,7.664950
Number of stations,2.307899,0.845851
Price,2293.848245,1512.964056


## Data normalization

In [14]:
normalized_features = np.array(train_features[['Latitude', 'Longitude', 'Area m2', 'Balcony m2', 'Construction year', 'Floor', 'Highest floor', 'Closest station (min)', 'Number of stations' ]])

In [15]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [16]:
#normalizer.adapt(normalized_features)
normalizer.adapt(np.array(train_features))

In [ ]:
print(normalizer.mean.numpy())

In [ ]:
#first = np.array(train_features[['Latitude', 'Longitude', 'Area m2', 'Balcony m2', 'Construction year', 'Floor', 'Highest floor', 'Closest station (min)', 'Number of stations' ]][:1])
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

## DNN settings

In [17]:
def build_and_compile_model(norm, loss, learning_rate, do_ratio):
    model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dropout(rate=do_ratio), 
      layers.Dense(64, activation='relu'),
      layers.Dropout(rate=do_ratio),
      layers.Dense(1)
    ])

    model.compile(loss=loss,optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model

In [20]:
f_loss=['mean_absolute_percentage_error']
l_rate = [0.001]
dropout_ratio = [0.1, 0.2, 0.3]

In [ ]:
nb_epochs=5000
for loss in f_loss:
    for rate in l_rate:
        for do in dropout_ratio:
            model_name = 'fudosan_{}_lr_{}_dropout_{}_epochs_{}.dnn'.format(loss, rate, do, nb_epochs)
            print(model_name)

In [21]:
%%time
import pickle

nb_epochs=5000
for loss in f_loss:
    for rate in l_rate:
        for do in dropout_ratio:
            price_reduction_model = build_and_compile_model(normalizer, loss, rate, do)
            history = price_reduction_model.fit(
            train_features,
            train_labels,
            validation_split=0.2,
            verbose=2, epochs=nb_epochs)
            price_reduction_model.save('minimal_{}_lr_{}_dropout_{}_epochs_{}'.format(loss, rate, do, nb_epochs))
            with open('history_minimal_{}_lr_{}_dropout_{}_epochs_{}'.format(loss, rate, do, nb_epochs), 'wb') as file_pi:
                pickle.dump(history.history, file_pi)

Epoch 1/5000
177/177 - 2s - loss: 97.7486 - val_loss: 91.9085 - 2s/epoch - 9ms/step
Epoch 2/5000
177/177 - 1s - loss: 81.7999 - val_loss: 73.4354 - 518ms/epoch - 3ms/step
Epoch 3/5000
177/177 - 0s - loss: 60.8978 - val_loss: 48.1884 - 485ms/epoch - 3ms/step
Epoch 4/5000
177/177 - 1s - loss: 40.0377 - val_loss: 32.7805 - 517ms/epoch - 3ms/step
Epoch 5/5000
177/177 - 1s - loss: 32.1519 - val_loss: 29.8233 - 501ms/epoch - 3ms/step
Epoch 6/5000
177/177 - 0s - loss: 30.2529 - val_loss: 28.7850 - 500ms/epoch - 3ms/step
Epoch 7/5000
177/177 - 0s - loss: 29.0672 - val_loss: 27.8598 - 499ms/epoch - 3ms/step
Epoch 8/5000
177/177 - 0s - loss: 28.5517 - val_loss: 27.2379 - 485ms/epoch - 3ms/step
Epoch 9/5000
177/177 - 0s - loss: 27.7948 - val_loss: 26.8756 - 498ms/epoch - 3ms/step
Epoch 10/5000
177/177 - 0s - loss: 27.4823 - val_loss: 26.6611 - 498ms/epoch - 3ms/step
Epoch 11/5000
177/177 - 0s - loss: 27.2235 - val_loss: 26.4261 - 499ms/epoch - 3ms/step
Epoch 12/5000
177/177 - 0s - loss: 26.9808 -

Epoch 95/5000
177/177 - 1s - loss: 23.4531 - val_loss: 22.8117 - 589ms/epoch - 3ms/step
Epoch 96/5000
177/177 - 1s - loss: 23.1681 - val_loss: 22.8593 - 526ms/epoch - 3ms/step
Epoch 97/5000
177/177 - 1s - loss: 23.2090 - val_loss: 22.9093 - 541ms/epoch - 3ms/step
Epoch 98/5000
177/177 - 0s - loss: 23.0000 - val_loss: 22.8597 - 493ms/epoch - 3ms/step
Epoch 99/5000
177/177 - 1s - loss: 22.9713 - val_loss: 22.7912 - 527ms/epoch - 3ms/step
Epoch 100/5000
177/177 - 1s - loss: 23.2236 - val_loss: 22.7415 - 570ms/epoch - 3ms/step
Epoch 101/5000
177/177 - 0s - loss: 22.9786 - val_loss: 22.6910 - 498ms/epoch - 3ms/step
Epoch 102/5000
177/177 - 0s - loss: 23.1853 - val_loss: 22.7156 - 499ms/epoch - 3ms/step
Epoch 103/5000
177/177 - 1s - loss: 23.0813 - val_loss: 22.6721 - 502ms/epoch - 3ms/step
Epoch 104/5000
177/177 - 0s - loss: 23.0197 - val_loss: 22.6805 - 499ms/epoch - 3ms/step
Epoch 105/5000
177/177 - 0s - loss: 23.1180 - val_loss: 22.6811 - 500ms/epoch - 3ms/step
Epoch 106/5000
177/177 - 0

177/177 - 1s - loss: 22.2433 - val_loss: 21.7964 - 500ms/epoch - 3ms/step
Epoch 188/5000
177/177 - 0s - loss: 22.3126 - val_loss: 21.7541 - 484ms/epoch - 3ms/step
Epoch 189/5000
177/177 - 0s - loss: 22.2926 - val_loss: 21.7700 - 484ms/epoch - 3ms/step
Epoch 190/5000
177/177 - 0s - loss: 22.0839 - val_loss: 21.8290 - 484ms/epoch - 3ms/step
Epoch 191/5000
177/177 - 0s - loss: 22.2467 - val_loss: 21.7990 - 499ms/epoch - 3ms/step
Epoch 192/5000
177/177 - 1s - loss: 22.2655 - val_loss: 21.7035 - 500ms/epoch - 3ms/step
Epoch 193/5000
177/177 - 1s - loss: 22.2856 - val_loss: 21.7895 - 515ms/epoch - 3ms/step
Epoch 194/5000
177/177 - 0s - loss: 22.0673 - val_loss: 21.7424 - 487ms/epoch - 3ms/step
Epoch 195/5000
177/177 - 1s - loss: 22.2010 - val_loss: 21.7592 - 500ms/epoch - 3ms/step
Epoch 196/5000
177/177 - 0s - loss: 22.1802 - val_loss: 21.6885 - 489ms/epoch - 3ms/step
Epoch 197/5000
177/177 - 1s - loss: 22.2146 - val_loss: 21.7679 - 501ms/epoch - 3ms/step
Epoch 198/5000
177/177 - 1s - loss: 

Epoch 280/5000
177/177 - 1s - loss: 21.7378 - val_loss: 21.2085 - 635ms/epoch - 4ms/step
Epoch 281/5000
177/177 - 1s - loss: 21.7474 - val_loss: 21.1120 - 649ms/epoch - 4ms/step
Epoch 282/5000
177/177 - 1s - loss: 21.5913 - val_loss: 21.0747 - 632ms/epoch - 4ms/step
Epoch 283/5000
177/177 - 1s - loss: 21.7519 - val_loss: 21.1425 - 618ms/epoch - 3ms/step
Epoch 284/5000
177/177 - 1s - loss: 21.8661 - val_loss: 21.0460 - 634ms/epoch - 4ms/step
Epoch 285/5000
177/177 - 1s - loss: 21.4498 - val_loss: 21.1004 - 646ms/epoch - 4ms/step
Epoch 286/5000
177/177 - 1s - loss: 21.7846 - val_loss: 21.1449 - 634ms/epoch - 4ms/step
Epoch 287/5000
177/177 - 1s - loss: 21.7574 - val_loss: 21.1591 - 646ms/epoch - 4ms/step
Epoch 288/5000
177/177 - 1s - loss: 21.6515 - val_loss: 21.0949 - 650ms/epoch - 4ms/step
Epoch 289/5000
177/177 - 1s - loss: 21.7758 - val_loss: 21.0262 - 632ms/epoch - 4ms/step
Epoch 290/5000
177/177 - 1s - loss: 21.6717 - val_loss: 21.1085 - 633ms/epoch - 4ms/step
Epoch 291/5000
177/17

177/177 - 1s - loss: 21.3912 - val_loss: 20.6241 - 650ms/epoch - 4ms/step
Epoch 373/5000
177/177 - 1s - loss: 21.1158 - val_loss: 20.6429 - 680ms/epoch - 4ms/step
Epoch 374/5000
177/177 - 1s - loss: 21.4064 - val_loss: 20.6968 - 652ms/epoch - 4ms/step
Epoch 375/5000
177/177 - 1s - loss: 21.1207 - val_loss: 20.5918 - 650ms/epoch - 4ms/step
Epoch 376/5000
177/177 - 1s - loss: 21.3502 - val_loss: 20.6731 - 650ms/epoch - 4ms/step
Epoch 377/5000
177/177 - 1s - loss: 21.0155 - val_loss: 20.6393 - 650ms/epoch - 4ms/step
Epoch 378/5000
177/177 - 1s - loss: 20.9352 - val_loss: 20.6328 - 649ms/epoch - 4ms/step
Epoch 379/5000
177/177 - 1s - loss: 21.0869 - val_loss: 20.5283 - 651ms/epoch - 4ms/step
Epoch 380/5000
177/177 - 1s - loss: 21.2749 - val_loss: 20.5330 - 651ms/epoch - 4ms/step
Epoch 381/5000
177/177 - 1s - loss: 21.3123 - val_loss: 20.4941 - 652ms/epoch - 4ms/step
Epoch 382/5000
177/177 - 1s - loss: 21.1297 - val_loss: 20.5039 - 648ms/epoch - 4ms/step
Epoch 383/5000
177/177 - 1s - loss: 

...

177/177 - 1s - loss: 22.4284 - val_loss: 20.2657 - 722ms/epoch - 4ms/step
Epoch 4216/5000
177/177 - 1s - loss: 22.2796 - val_loss: 20.3284 - 722ms/epoch - 4ms/step
Epoch 4217/5000
177/177 - 1s - loss: 22.5484 - val_loss: 20.2659 - 728ms/epoch - 4ms/step
Epoch 4218/5000
177/177 - 1s - loss: 22.5416 - val_loss: 20.3027 - 716ms/epoch - 4ms/step
Epoch 4219/5000
177/177 - 1s - loss: 22.1601 - val_loss: 20.2704 - 722ms/epoch - 4ms/step
Epoch 4220/5000
177/177 - 1s - loss: 22.3620 - val_loss: 20.4534 - 721ms/epoch - 4ms/step
Epoch 4221/5000
177/177 - 1s - loss: 22.4502 - val_loss: 20.3569 - 706ms/epoch - 4ms/step
Epoch 4222/5000
177/177 - 1s - loss: 22.2866 - val_loss: 20.2587 - 721ms/epoch - 4ms/step
Epoch 4223/5000
177/177 - 1s - loss: 22.1297 - val_loss: 20.4431 - 716ms/epoch - 4ms/step
Epoch 4224/5000
177/177 - 1s - loss: 22.4299 - val_loss: 20.2645 - 722ms/epoch - 4ms/step
Epoch 4225/5000
177/177 - 1s - loss: 22.1219 - val_loss: 20.4339 - 722ms/epoch - 4ms/step
Epoch 4226/5000
177/177 - 

Epoch 4307/5000
177/177 - 1s - loss: 22.5249 - val_loss: 20.2361 - 722ms/epoch - 4ms/step
Epoch 4308/5000
177/177 - 1s - loss: 22.5529 - val_loss: 20.2089 - 722ms/epoch - 4ms/step
Epoch 4309/5000
177/177 - 1s - loss: 22.3258 - val_loss: 20.1861 - 726ms/epoch - 4ms/step
Epoch 4310/5000
177/177 - 1s - loss: 22.0069 - val_loss: 20.3584 - 718ms/epoch - 4ms/step
Epoch 4311/5000
177/177 - 1s - loss: 22.0565 - val_loss: 20.2896 - 722ms/epoch - 4ms/step
Epoch 4312/5000
177/177 - 1s - loss: 21.9860 - val_loss: 20.2970 - 722ms/epoch - 4ms/step
Epoch 4313/5000
177/177 - 1s - loss: 22.3875 - val_loss: 20.3826 - 722ms/epoch - 4ms/step
Epoch 4314/5000
177/177 - 1s - loss: 22.1810 - val_loss: 20.1890 - 725ms/epoch - 4ms/step
Epoch 4315/5000
177/177 - 1s - loss: 22.5705 - val_loss: 20.2521 - 719ms/epoch - 4ms/step
Epoch 4316/5000
177/177 - 1s - loss: 22.2335 - val_loss: 20.1208 - 722ms/epoch - 4ms/step
Epoch 4317/5000
177/177 - 1s - loss: 22.3326 - val_loss: 20.1590 - 721ms/epoch - 4ms/step
Epoch 4318

177/177 - 1s - loss: 22.5228 - val_loss: 20.2789 - 722ms/epoch - 4ms/step
Epoch 4399/5000
177/177 - 1s - loss: 22.3814 - val_loss: 20.4481 - 721ms/epoch - 4ms/step
Epoch 4400/5000
177/177 - 1s - loss: 22.5511 - val_loss: 20.3651 - 722ms/epoch - 4ms/step
Epoch 4401/5000
177/177 - 1s - loss: 22.4802 - val_loss: 20.2881 - 722ms/epoch - 4ms/step
Epoch 4402/5000
177/177 - 1s - loss: 22.2364 - val_loss: 20.0810 - 769ms/epoch - 4ms/step
Epoch 4403/5000
177/177 - 1s - loss: 22.3259 - val_loss: 20.2076 - 711ms/epoch - 4ms/step
Epoch 4404/5000
177/177 - 1s - loss: 22.2104 - val_loss: 20.1989 - 707ms/epoch - 4ms/step
Epoch 4405/5000
177/177 - 1s - loss: 22.3928 - val_loss: 20.1937 - 722ms/epoch - 4ms/step
Epoch 4406/5000
177/177 - 1s - loss: 22.3248 - val_loss: 20.1536 - 721ms/epoch - 4ms/step
Epoch 4407/5000
177/177 - 1s - loss: 22.6820 - val_loss: 20.1580 - 723ms/epoch - 4ms/step
Epoch 4408/5000
177/177 - 1s - loss: 22.1258 - val_loss: 20.2909 - 722ms/epoch - 4ms/step
Epoch 4409/5000
177/177 - 

Epoch 4490/5000
177/177 - 1s - loss: 22.3131 - val_loss: 20.1559 - 722ms/epoch - 4ms/step
Epoch 4491/5000
177/177 - 1s - loss: 22.6732 - val_loss: 20.3835 - 722ms/epoch - 4ms/step
Epoch 4492/5000
177/177 - 1s - loss: 22.4458 - val_loss: 20.2875 - 723ms/epoch - 4ms/step
Epoch 4493/5000
177/177 - 1s - loss: 22.5048 - val_loss: 20.4655 - 722ms/epoch - 4ms/step
Epoch 4494/5000
177/177 - 1s - loss: 22.1176 - val_loss: 20.1932 - 722ms/epoch - 4ms/step
Epoch 4495/5000
177/177 - 1s - loss: 22.0997 - val_loss: 20.2717 - 721ms/epoch - 4ms/step
Epoch 4496/5000
177/177 - 1s - loss: 22.2036 - val_loss: 20.2296 - 722ms/epoch - 4ms/step
Epoch 4497/5000
177/177 - 1s - loss: 22.3828 - val_loss: 20.2213 - 702ms/epoch - 4ms/step
Epoch 4498/5000
177/177 - 1s - loss: 22.1140 - val_loss: 20.2831 - 707ms/epoch - 4ms/step
Epoch 4499/5000
177/177 - 1s - loss: 22.1790 - val_loss: 20.3427 - 737ms/epoch - 4ms/step
Epoch 4500/5000
177/177 - 1s - loss: 22.3766 - val_loss: 20.2478 - 730ms/epoch - 4ms/step
Epoch 4501

177/177 - 1s - loss: 22.2805 - val_loss: 20.1455 - 722ms/epoch - 4ms/step
Epoch 4582/5000
177/177 - 1s - loss: 22.1981 - val_loss: 20.2035 - 722ms/epoch - 4ms/step
Epoch 4583/5000
177/177 - 1s - loss: 22.1552 - val_loss: 20.3088 - 730ms/epoch - 4ms/step
Epoch 4584/5000
177/177 - 1s - loss: 22.2829 - val_loss: 20.4294 - 714ms/epoch - 4ms/step
Epoch 4585/5000
177/177 - 1s - loss: 22.2811 - val_loss: 20.3269 - 722ms/epoch - 4ms/step
Epoch 4586/5000
177/177 - 1s - loss: 22.6160 - val_loss: 20.1806 - 706ms/epoch - 4ms/step
Epoch 4587/5000
177/177 - 1s - loss: 22.2957 - val_loss: 20.1429 - 706ms/epoch - 4ms/step
Epoch 4588/5000
177/177 - 1s - loss: 22.3753 - val_loss: 20.1445 - 706ms/epoch - 4ms/step
Epoch 4589/5000
177/177 - 1s - loss: 22.3629 - val_loss: 20.3206 - 710ms/epoch - 4ms/step
Epoch 4590/5000
177/177 - 1s - loss: 22.5877 - val_loss: 20.3712 - 718ms/epoch - 4ms/step
Epoch 4591/5000
177/177 - 1s - loss: 22.1642 - val_loss: 20.2045 - 722ms/epoch - 4ms/step
Epoch 4592/5000
177/177 - 

In [ ]:
price_reduction_model = build_and_compile_model(normalizer)

In [ ]:
price_reduction_model.summary()

In [ ]:
%%time
history = price_reduction_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=2, epochs=5000)

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [Price]')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
print(history.history['val_loss'][-1])
print(history.history['loss'][-1])

In [ ]:
test_results = {}

In [ ]:
test_results['mape_a_0.01_dropout_0.2_e_10000'] = price_reduction_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
test_results

In [ ]:
test_labels.describe()

In [ ]:
predictions = price_reduction_model.predict(test_features)
true_price = np.array(test_labels).reshape(-1,1)
#predictions = price_reduction_model.predict(train_features)
#true_price = np.array(train_labels).reshape(-1,1)
error = abs(predictions-true_price)
print(error.sum()/error.shape[0])
#test_features.iloc[0]

In [ ]:
test_predictions = price_reduction_model.predict(test_features).flatten()
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Price Error [man]')
_ = plt.ylabel('Count')

In [ ]:
np.max(test_predictions)

In [ ]:
for i, p in enumerate(test_predictions):
    if p>100000:
        print(i, test_predictions[i], test_labels.iloc[i])

In [ ]:
print(test_features.iloc[202])
print(test_labels.iloc[202])

In [ ]:
price_reduction_model.save('price_estimation_loss_mae_a_0.01_dropout_0.0_e_1000')

In [ ]:
model_names= [name for name in os.listdir() if 'fudosan_mean' in name]
model_names

In [ ]:
test_features = test_features.drop([1154,5131])
test_labels = test_labels.drop([1154,5131])
for name in model_names:
    print('\nEvaluating model {}:'.format(name))
    model = keras.models.load_model(name)
    #test_predictions = model.predict(test_features).flatten()
    #for i, p in enumerate(test_predictions):
    #    if p>100000:
    #        print(i, test_predictions[i], test_labels.iloc[i])
    print('Training error: {}'.format(model.evaluate(train_features, train_labels, verbose=0)))
    print('Test error: {}'.format(model.evaluate(test_features, test_labels, verbose=0)))
    

In [ ]:
#data.loc[data.index==id].index
test_features.iloc[202]

In [ ]:
for name in ['fudosan_mean_squared_error_lr_0.001_dropout_0.1_epochs_5000.dnn']:
    print('\nEvaluating model {}:'.format(name))
    model = keras.models.load_model(name)
    training_predictions = model.predict(train_features).flatten()
    training_predictions = training_predictions.reshape(-1,1)
    true_price = np.array(train_labels).reshape(-1,1)
    training_error = abs(training_predictions - true_price)
    r_training_error = training_error/true_price
    r_training_error  = r_training_error.sum()/r_training_error.shape[0] *100
    
    
    test_predictions = model.predict(test_features).flatten()
    test_predictions = test_predictions.reshape(-1,1)
    true_price = np.array(test_labels).reshape(-1,1)
    test_error = abs(test_predictions - true_price)
    r_test_error = test_error/true_price
    r_test_error = r_test_error.sum() / r_test_error.shape[0]*100
    
    print('Training error in %: {}'.format(r_training_error))
    print('Test error in : {}'.format(r_test_error))
    

In [ ]:
training_predictions = model.predict(train_features).flatten()
training_predictions=training_predictions.reshape(-1,1)
training_predictions.shape

In [ ]:
true_price = np.array(train_labels).reshape(-1,1)
true_price.shape

In [ ]:
error = abs(training_predictions-true_price)
r_error = error/true_price
r_error.sum()/r_error.shape[0]*100